# Introduction

[<img align="center" src="https://colab.research.google.com/assets/colab-badge.svg" />](https://colab.research.google.com/github/marshmellow77/automated-prompt-engineering/blob/main/automated-prompt-engineering.ipynb)


This notebook demonstrates how to use Google's Gemini model to automate prompt engineering.

Prompt engineering is a powerful way to improve the responses og large language models (LLMs). Bit it is also a manual, tedious, iterative process and it quickly accumulates technical debt and waste since each handcrafted prompt is specific to a model (and its version) as well as the task at hand.

In this notebook we will learn how to use the DSPy library to autonomously and automatically create prompts that are optimised for a specific model and the task at hand.


# Manual Prompt Engineering

Manual prompt engineering is very tedious - let's look at an example where we carefully handcraft a prompt for our task and model.

## Setup

In [1]:
# As of 3 April 2024, VertexAI is not yet integrated into DSPy. But there already exists a PR for it which we can leverage.
!pip install -U git+https://github.com/marshmellow77/dspy.git@seedstart-random-search#egg=dspy-ai

  Cloning https://github.com/marshmellow77/dspy.git (to revision seedstart-random-search) to /tmp/pip-install-4b67w7yz/dspy-ai_dc07ac251c49487c898a2cb09653e4f7
  Running command git clone --filter=blob:none --quiet https://github.com/marshmellow77/dspy.git /tmp/pip-install-4b67w7yz/dspy-ai_dc07ac251c49487c898a2cb09653e4f7
  Running command git checkout -q seedstart-random-search
  error: pathspec 'seedstart-random-search' did not match any file(s) known to git
  error: subprocess-exited-with-error
  
  × git checkout -q seedstart-random-search did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git checkout -q seedstart-random-search did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
!pip install --upgrade google-cloud-aiplatform
!pip install Jinja2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 52.7 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.73.0
    Uninstalling google-cloud-aiplatform-1.73.0:
      Successfully uninstalled google-cloud-aiplatform-1.73.0


In [1]:
import os
import sys

IS_COLAB = "google.colab" in sys.modules
if not IS_COLAB:
    raise ValueError("This notebook should be run using Google Colab.")

if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

In [2]:
import vertexai

project_id = "my-project-269206"
vertexai.init(project=project_id)

In [3]:
from vertexai.generative_models import GenerativeModel

gemini_pro = GenerativeModel("gemini-1.0-pro")

## Zero shot attempt

Let's first try to use Gemini Pro for a mathematical text question

In [4]:
prompt = """Given the fields `question`, produce the fields `answer`.

Question: Heather is going to sew 150 aprons that are to be used for a kiddie crew program.
She already was able to sew 13 aprons, and today, she sewed three times as many aprons.
How many aprons should she sew tomorrow if she wants to sew half of the remaining number of aprons needed?

Answer:"""

# The correct answer is 49.

In [5]:
config = {"temperature": 0.1}

In [6]:
response = gemini_pro.generate_content(contents=prompt, generation_config=config)
print(response.text)

## Answer

Heather needs to sew a total of 150 aprons for the kiddie crew program. She has already sewn 13 aprons, and today, she sewed three times as many, which is 13 x 3 = 39 aprons.

Therefore, she has sewn a total of 13 + 39 = 52 aprons so far.

The remaining number of aprons needed is 150 - 52 = 98 aprons.

Heather wants to sew half of the remaining number of aprons tomorrow, which is 98 / 2 = 49 aprons.

Therefore, Heather should sew **49 aprons** tomorrow to complete half of the remaining number of aprons needed. 



We can see that Gemini Pro got this one wrong. Let's use best practices including Chain of thought and few shot prompting to improve Gemini's performance!

## Few shot prompting with Chain of Thought

In [7]:
prompt = """Given the fields `question`, produce the fields `answer`.

---

Follow the following format.

Question: <Question>
Rationale: Let's think step by step ...
Answer: <Answer>

---

Question: A gumball machine has red, green, and blue gumballs. The machine has half as many blue gumballs as red gumballs.
For each blue gumball, the machine has 4 times as many green gumballs. If the machine has 16 red gumballs how many gumballs are in the machine?
Rationale: Let's think step by step.
First, we can find the number of blue gumballs in the machine.
Since the machine has half as many blue gumballs as red gumballs, and there are 16 red gumballs, there must be 16 / 2 = 8 blue gumballs.
Next, we can find the number of green gumballs in the machine.
Since the machine has 4 times as many green gumballs as blue gumballs, there must be 8 x 4 = 32 green gumballs.
Finally, we can add up the number of red, blue, and green gumballs to find the total number of gumballs in the machine: 16 + 8 + 32 = 56.
Answer: 56

---

Question: Rachel makes $12.00 as a waitress in a coffee shop. In one hour, she serves 20 different people and they all leave her a $1.25 tip. How much money did she make in that hour?
Rationale: Let's think step by step.
First, we need to find out how much money Rachel made from tips. She served 20 people and each person left her a $1.25 tip, so she made 20 * $1.25 = $25.00 in tips.
Next, we need to add her hourly wage to the money she made from tips to find out how much money she made in total. She made $12.00 per hour, so in one hour she made $12.00 + $25.00 = $37.00.
Answer: 37

---

Question: Heather is going to sew 150 aprons that are to be used for a kiddie crew program. She already was able to sew 13 aprons, and today, she sewed three times as many aprons. How many aprons should she sew tomorrow if she wants to sew half of the remaining number of aprons needed?
Rationale:"""

In [8]:
response = gemini_pro.generate_content(contents=prompt, generation_config=config)
print(response.text)

Let's think step by step.

Heather already sewed 13 aprons, and today she sewed three times as many, so she sewed 3 * 13 = 39 aprons today.

In total, she has already sewn 13 + 39 = 52 aprons.

She needs to sew 150 aprons in total, so she still needs to sew 150 - 52 = 98 aprons.

If she wants to sew half of the remaining number of aprons needed tomorrow, she needs to sew 98 / 2 = 49 aprons tomorrow.

Answer: 49


Nice, this worked!

Now we have a good a good prompt for our model and the task at hand (mathematical text questions). But there are a few issues:
* Our prompt works well on our model, but what if we want to use another model or another version (e.g. Gemini Ultra of Gemini 1.5)? Will it still work for those models?
* We had to develop a few examples, and especially coming up with the rationale for each example was tedious

The question is, could we automate this process so that next time we need to repeat this exercise we can just automatically create few shot examples that are optimised for our model and the task at hand?

# Automated prompt engineering with DSPy

DSPy is a library that allows us to automate this process. Let's see how it works.

## Setup

In [19]:
pip install dspy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.2/345.2 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/9

In [9]:
import dspy

In [10]:
dspy_gemini_pro = dspy.GoogleVertexAI(
    "gemini-1.0-pro",
    temperature=0,
)

dspy.settings.configure(lm=dspy_gemini_pro)

## Dataset

We will use the [GSM8K dataset](https://paperswithcode.com/dataset/gsm8k) which consists of inguistically diverse grade school math word problems.

In [11]:
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

gms8k = GSM8K()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.94k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

100%|██████████| 1319/1319 [00:00<00:00, 25629.53it/s]


In [12]:
train, val, test = gms8k.train[:60], gms8k.dev[:20], gms8k.test[:20]

In [13]:
train[0]

Example({'question': "The result from the 40-item Statistics exam Marion and Ella took already came out. Ella got 4 incorrect answers while Marion got 6 more than half the score of Ella. What is Marion's score?", 'gold_reasoning': "Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items.", 'answer': '24'}) (input_keys={'question'})

In [14]:
train[0].gold_reasoning

"Ella's score is 40 items - 4 items = <<40-4=36>>36 items. Half of Ella's score is 36 items / 2 = <<36/2=18>>18 items. So, Marion's score is 18 items + 6 items = <<18+6=24>>24 items."

We can see that the dataset has a field `gold_resoning`, which already provides reasoning. Since this is what we want to automate, let's delete these for the training and validation datasets.

In [15]:
# Iterate through datasets and modify the dicts
for dataset in [train, val]:
    for example in dataset:
        example["gold_reasoning"] = ""

In [16]:
train[0].gold_reasoning

''

## Defining the signature

Signatures allow you tell the LM what it needs to do, rather than specify how we should ask the LM to do it.

In [17]:
class GSM8KSignature(dspy.Signature):
    """Answer math problems with numbers or short phrases."""

    question = dspy.InputField()
    answer = dspy.OutputField(desc="Usually a number or short phrase.")

Now we can use this signature to run a test with Gemini.

In [18]:
generate_answer = dspy.Predict(GSM8KSignature)
pred = generate_answer(question=test[0].question)

print(f"Question: {test[0].question}")
print(f"Predicted Answer: {pred.answer}")
print(f"Actual Answer: {test[0].answer}")

 		You are using the client GoogleVertexAI, which will be removed in DSPy 2.6.
 		Changing the client is straightforward and will let you use new features (Adapters) that improve the consistency of LM outputs, especially when using chat LMs. 

 		Learn more about the changes and how to migrate at
 		https://github.com/stanfordnlp/dspy/blob/main/examples/migration.ipynb


Question: Amber, Micah, and Ahito ran 52 miles in total. Amber ran 8 miles. Micah ran 3.5 times what Amber ran. How many miles did Ahito run?
Predicted Answer: Question: Amber, Micah, and Ahito ran 52 miles in total. Amber ran 8 miles. Micah ran 3.5 times what Amber ran. How many miles did Ahito run?
Answer: 28 miles
Actual Answer: 16


In [19]:
dspy_gemini_pro.inspect_history(n=1)




Answer math problems with numbers or short phrases.

---

Follow the following format.

Question: ${question}
Answer: Usually a number or short phrase.

---

Question: Amber, Micah, and Ahito ran 52 miles in total. Amber ran 8 miles. Micah ran 3.5 times what Amber ran. How many miles did Ahito run?
Answer:Question: Amber, Micah, and Ahito ran 52 miles in total. Amber ran 8 miles. Micah ran 3.5 times what Amber ran. How many miles did Ahito run?
Answer: 28 miles





'\n\n\nAnswer math problems with numbers or short phrases.\n\n---\n\nFollow the following format.\n\nQuestion: ${question}\nAnswer: Usually a number or short phrase.\n\n---\n\nQuestion: Amber, Micah, and Ahito ran 52 miles in total. Amber ran 8 miles. Micah ran 3.5 times what Amber ran. How many miles did Ahito run?\nAnswer:\x1bQuestion: Amber, Micah, and Ahito ran 52 miles in total. Amber ran 8 miles. Micah ran 3.5 times what Amber ran. How many miles did Ahito run?\nAnswer: 28 miles\x1b\n\n\n'

Similar to above Gemini didn't get this one right. Let's evaluate Gemini of the test dataset to establish a baseline.

## Model evaluation with zero shot

To run the evaluation programmatically we define a DSPy module These modules abstract a prompting technique (like chain of thought or ReAct). Crucially, they are generalized to handle any DSPy Signature.

In [20]:
class GSM8KModule(dspy.Module):
    def __init__(self):
        super().__init__()
        # here we use the dspy.Predict module which uses zero shot prompting to generate answers
        self.prog = dspy.Predict(GSM8KSignature)

    def forward(self, question):
        return self.prog(question=question)

In [21]:
gsm8k_zero_shot = GSM8KModule()

In [22]:
from dspy.evaluate import Evaluate

NUM_THREADS = 4 # number of threads to use for parallel processing
evaluate = Evaluate(
    devset=test, # the test set
    metric=gsm8k_metric, # the metric to use -> this will convert responses to integers to compare with the gold answers
    num_threads=NUM_THREADS,
    display_progress=True,
    display_table=20, # how many rows to display
)

In [23]:
evaluate(gsm8k_zero_shot)

Average Metric: 3.00 / 20 (15.0%): 100%|██████████| 20/20 [00:05<00:00,  3.82it/s]

2024/12/14 06:31:24 INFO dspy.evaluate.evaluate: Average Metric: 3 / 20 (15.0%)


,question,gold_reasoning,example_answer,pred_answer,gsm8k_metric
0,"Amber, Micah, and Ahito ran 52 miles in total. Amber ran 8 miles. ...",Amber ran <<8=8>>8 miles. Micah ran 3.5 * 8 miles = <<3.5*8=28>>28...,16,"Question: Amber, Micah, and Ahito ran 52 miles in total. Amber ran...",
1,Miguel uses 2 pads of paper a week for his drawing. If there are 3...,Miguel uses 30 x 2 = <<30*2=60>>60 sheets of paper every week. The...,240,Question: Miguel uses 2 pads of paper a week for his drawing. If t...,
2,"At a certain grade level, three-fourths of students have a desktop...",Twenty students represent 1 - 3/4 = 1/4 of the students at that le...,80,"Question: At a certain grade level, three-fourths of students have...",
3,Comet Halley orbits the sun every 75 years. Bill's dad saw the Com...,Bill saw the Comet for the second time when he was 30 years * 3= <...,15,Answer: 60 years old,
4,Tom plants 10 trees a year. Every year he also chops down 2 trees ...,He gets 10-2=<<10-2=8>>8 new trees a year After 10 years he has 8*...,91,Answer: 100 trees. Here's the breakdown: * Starts with 50 trees. *...,
5,John picks 4 bananas on Wednesday. Then he picks 6 bananas on Thur...,"Combining Wednesday and Thursday, John has 4 bananas + 6 bananas =...",22,Answer: 22,✔️ [True]
6,Peyton scheduled after-work activities of a one hour yoga class on...,Peyton’s cooking class will last 3 * 1 = <<3*1=3>>3 hours. The mus...,8,Question: Peyton scheduled after-work activities of a one hour yog...,
7,Ben has 4 tubes of blue paint and 3 tubes of yellow paint. Jasper ...,Jasper has 4/2= <<4/2=2>>2 tubes of blue paint Jasper has 3*3=<<3*...,11,Answer: 10,
8,Elaina is holding the final concert in her tour. To celebrate her ...,"The concert, minus the encore, lasted for 65-minute concert – 15-m...",25,Answer: 25 minutes,✔️ [True]
9,Hannah slips on a banana peel and breaks her arm. The doctor charg...,First find the length of the visit in hours: 30 minutes / 60 minut...,482,$200 + $300/hour * 0.5 hours + $4/pill * 30 pills + $6/hour * 2 ho...,


15.0

# Bootstrapping few shot examples

Now we will leverage Gemini Ultra to bootstrap few shot examples which will (hopefully) improve Gemini Pro's performance on the test dataset. With Gemini Ultra we will create a few reasoning examples which we can include in the prompt that we will eventually send to Gemini Pro. Ultra will produce a few candidates and test them on a validation dataset using the `gsm8k_metric`, i.e. the metric we want to optimise for. Once the best candidates have been identified these examples will then be used to create a few shot prompt.

First we define a Chain of Thought module:

In [24]:
class ZeroShotCoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(
            GSM8KSignature,
        )

    def forward(self, question):
        return self.prog(question=question)

In [25]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

Now we can start the bootstrapping:

In [ ]:
from datetime import datetime

RUN_FROM_SCRATCH = True
bootstrapped_demos = 8 # how many examples are randomly being used from the training dataset
labeled_demos = 3 # how many examples will be in final prompt
candidate_programs = 2 # how many candidates will be created and evaluated (equivalent to epochs)
teacher_model_id = "gemini-1.0-pro"

if RUN_FROM_SCRATCH:
    dspy_gemini_ultra = dspy.GoogleVertexAI(
        teacher_model_id,
        temperature=0,
    )
    dspy.settings.configure(lm=dspy_gemini_ultra, timeout=0)
    config = dict(
        max_bootstrapped_demos=bootstrapped_demos,
        max_labeled_demos=labeled_demos,
        num_candidate_programs=candidate_programs,
        num_threads=4,
        stop_at_score=100.0,
    )
    bootstrap_optimizer = BootstrapFewShotWithRandomSearch(
        metric=gsm8k_metric, **config
    )
    cot_fewshot = bootstrap_optimizer.compile(ZeroShotCoT(), trainset=train, valset=val)

    # save the bootstrap demonstrations for future use
    timestamp_str = datetime.now().strftime("%Y%m%d-%H%M%S")
    filename = f"{timestamp_str}_{teacher_model_id}_{bootstrapped_demos}_{labeled_demos}_{candidate_programs}.json"
    cot_fewshot.save(filename)
else:
    cot_fewshot = ZeroShotCoT()
    cot_fewshot.load("20240403-173150_gemini-1.0-ultra_8_3_2.json")

Going to sample between 1 and 8 traces per predictor.
Will attempt to bootstrap 2 candidate sets.
Average Metric: 14.00 / 20 (70.0%): 100%|██████████| 20/20 [00:11<00:00,  1.77it/s]

2024/12/14 06:50:18 INFO dspy.evaluate.evaluate: Average Metric: 14 / 20 (70.0%)



New best score: 70.0 for seed -3
Scores so far: [70.0]
Best score so far: 70.0
Average Metric: 8.00 / 20 (40.0%): 100%|██████████| 20/20 [00:10<00:00,  1.94it/s]

2024/12/14 06:50:28 INFO dspy.evaluate.evaluate: Average Metric: 8 / 20 (40.0%)



Scores so far: [70.0, 40.0]
Best score so far: 70.0


 38%|███▊      | 23/60 [00:35<00:57,  1.55s/it]


Bootstrapped 8 full traces after 23 examples for up to 1 rounds, amounting to 23 attempts.
Average Metric: 16.00 / 20 (80.0%): 100%|██████████| 20/20 [00:08<00:00,  2.47it/s]

2024/12/14 06:51:12 INFO dspy.evaluate.evaluate: Average Metric: 16 / 20 (80.0%)



New best score: 80.0 for seed -1
Scores so far: [70.0, 40.0, 80.0]
Best score so far: 80.0


 17%|█▋        | 10/60 [00:17<01:21,  1.64s/it]

After this step we have our examples ready, and we can test Gemini Pro on the same test dataset as above.

In [ ]:
dspy.settings.configure(lm=dspy_gemini_pro, timeout=0)

In [ ]:
evaluate(cot_fewshot)

Nice, this improved Gemini Pro's performance significantly from 35% :)

In [ ]:
dspy_gemini_pro.inspect_history(n=1)